In [ ]:
import pandas as pd
import sqlalchemy
import os
from tqdm import tqdm
from dotenv import load_dotenv
import time  # Importar el módulo time para pausar

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales de la base de datos desde las variables de entorno
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

# Función para conectar a la base de datos
def connect_to_db():
    try:
        engine = sqlalchemy.create_engine(
            f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
        )
        connection = engine.connect()
        print("Conexión a la base de datos establecida con éxito.")
        return engine, connection
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        raise

# Conectar a MySQL utilizando las credenciales del archivo .env
engine, connection = connect_to_db()

# Ruta de la carpeta con los archivos
folder_path = 'D:\\GitHub\\Proyecto Final\\Proyecto_final_Henry\\Dataset'

# Columnas a mantener para green_tripdata
columns_to_keep_green = [
    'lpep_pickup_datetime', 'lpep_dropoff_datetime',
    'PULocationID', 'DOLocationID',
    'trip_distance', 'total_amount'
]

# Columnas a mantener para yellow_tripdata
columns_to_keep_yellow = [
    'tpep_pickup_datetime', 'tpep_dropoff_datetime',
    'PULocationID', 'DOLocationID',
    'trip_distance', 'total_amount'
]

# Columnas a mantener para fhv_tripdata
columns_to_keep_fhv = [
    'pickup_datetime', 'dropOff_datetime',
    'PUlocationID', 'DOlocationID',
    'Affiliated_base_number'
]

# Leer todos los archivos en la carpeta y estandarizar las columnas
for filename in os.listdir(folder_path):
    if filename.endswith('.parquet'):
        file_path = os.path.join(folder_path, filename)

        try:
            # Leer archivo .parquet
            print(f"\nLeyendo archivo {file_path}")
            df = pd.read_parquet(file_path)

            # Verificar si es green_tripdata, yellow_tripdata o fhv_tripdata
            if 'green' in filename.lower():
                columns_to_keep = columns_to_keep_green
                source_value = 'green_tripdata'
            elif 'yellow' in filename.lower():
                columns_to_keep = columns_to_keep_yellow
                source_value = 'yellow_tripdata'
            elif 'fhv' in filename.lower():
                columns_to_keep = columns_to_keep_fhv
                source_value = 'fhv_tripdata'
            else:
                print(f"El archivo {filename} no es ni 'green', 'yellow' ni 'fhv'. Saltando.")
                continue

            # Verificar si el archivo contiene todas las columnas necesarias
            if all(column in df.columns for column in columns_to_keep):
                print(f"El archivo {filename} contiene todas las columnas necesarias. Procesando...")

                # Seleccionar columnas necesarias
                df = df[columns_to_keep]

                # Renombrar columnas según sea necesario
                if 'yellow' in filename.lower():
                    df.rename(columns={
                        'tpep_pickup_datetime': 'Pickup_datetime',
                        'tpep_dropoff_datetime': 'DropOff_datetime',
                        'trip_distance': 'trip_miles',
                        'total_amount': 'driver_pay'
                    }, inplace=True)
                elif 'green' in filename.lower():
                    df.rename(columns={
                        'lpep_pickup_datetime': 'Pickup_datetime',
                        'lpep_dropoff_datetime': 'DropOff_datetime',
                        'trip_distance': 'trip_miles',
                        'total_amount': 'driver_pay'
                    }, inplace=True)
                elif 'fhv' in filename.lower():
                    df.rename(columns={
                        'pickup_datetime': 'Pickup_datetime',
                        'dropOff_datetime': 'DropOff_datetime',
                        'PUlocationID': 'PULocationID',
                        'DOlocationID': 'DOLocationID'
                    }, inplace=True)
                    df['trip_miles'] = 0.0  # Agregar columna trip_miles con valores por defecto
                    df['driver_pay'] = 0.0  # Agregar columna driver_pay con valores por defecto

                # Añadir la columna 'source'
                df['source'] = source_value

                # Calcular trip_time como la diferencia entre DropOff_datetime y Pickup_datetime
                df['Pickup_datetime'] = pd.to_datetime(df['Pickup_datetime'])
                df['DropOff_datetime'] = pd.to_datetime(df['DropOff_datetime'])
                df['trip_time'] = (df['DropOff_datetime'] - df['Pickup_datetime']).dt.total_seconds()

                # Dividir el DataFrame en bloques de 5,000 filas
                chunk_size = 150000  # Tamaño del chunk a 5,000 filas
                num_chunks = len(df) // chunk_size + 1
                chunk_counter = 0  # Contador de chunks procesados

                for i in tqdm(range(num_chunks), desc=f"Procesando {filename}", unit="chunk"):
                    chunk = df[i * chunk_size:(i + 1) * chunk_size]
                    try:
                        chunk.to_sql(name='taxi_fhv_data', con=engine, if_exists='append', index=False)
                        print(f"Chunk {i+1}/{num_chunks} del archivo {filename} cargado exitosamente.")
                    except sqlalchemy.exc.OperationalError as e:
                        print(f"Error al cargar chunk {i+1}: {e}. Intentando reconectar...")
                        engine, connection = connect_to_db()
                        chunk.to_sql(name='taxi_fhv_data', con=engine, if_exists='append', index=False)
                        print(f"Chunk {i+1} del archivo {filename} cargado exitosamente tras reconexión.")

                    chunk_counter += 1

                    # Cada 20 chunks, esperar 15 segundos y reconectar
                    if chunk_counter % 30 == 0:
                        print("Esperando 15 segundos...")
                        time.sleep(30)
                        print("Reconectando a la base de datos...")
                        connection.close()
                        engine.dispose()
                        engine, connection = connect_to_db()
                        print("Reconexión exitosa.")

                print(f"Datos del archivo {filename} cargados exitosamente.")
            else:
                print(f"El archivo {filename} no contiene todas las columnas necesarias. Saltando.")

        except FileNotFoundError:
            print(f"Archivo {file_path} no encontrado. Saltando.")
        except KeyError as e:
            print(f"Error de columna en {file_path}: {e}. Saltando.")
        except Exception as e:
            print(f"Error al procesar el archivo {filename}: {e}")


try:
    # Leer archivo .parquet
    df = pd.read_parquet(file_path)

    # Mostrar las columnas del archivo
    print(f"\nColumnas del archivo {file_to_inspect}:")
    print(df.columns.tolist())

except Exception as e:
    print(f"Error al leer {file_path}: {e}")

print("\nProceso de estandarización y carga completo.")


Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\kcasi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sqlalchemy\pool\base.py", line 986, in _finalize_fairy
    fairy._reset(
  File "C:\Users\kcasi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sqlalchemy\pool\base.py", line 1432, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\kcasi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sqlalchemy\engine\default.py", line 699, in do_rollback
    dbapi_connection.rollback()
  File "C:\Users\kcasi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pymysql\connections.py", line 493, in rollback
    self._read_ok_packet()
  File "C:\Users\kcasi\Ap

Conexión a la base de datos establecida con éxito.

Leyendo archivo D:\GitHub\Proyecto Final\Proyecto_final_Henry\Dataset\yellow_tripdata_2024-01.parquet
El archivo yellow_tripdata_2024-01.parquet contiene todas las columnas necesarias. Procesando...


Procesando yellow_tripdata_2024-01.parquet:   5%|▌         | 1/20 [00:22<07:07, 22.49s/chunk]

Chunk 1/20 del archivo yellow_tripdata_2024-01.parquet cargado exitosamente.


Procesando yellow_tripdata_2024-01.parquet:  10%|█         | 2/20 [00:40<05:52, 19.60s/chunk]

...

Chunk 8/25 del archivo yellow_tripdata_2024-05.parquet cargado exitosamente.


Procesando yellow_tripdata_2024-05.parquet:  36%|███▌      | 9/25 [08:07<14:52, 55.79s/chunk]

Chunk 9/25 del archivo yellow_tripdata_2024-05.parquet cargado exitosamente.


Procesando yellow_tripdata_2024-05.parquet:  40%|████      | 10/25 [10:32<20:51, 83.46s/chunk]

Chunk 10/25 del archivo yellow_tripdata_2024-05.parquet cargado exitosamente.
